In [2]:
import pandas as pd
import numpy as np
import matplotlib_inline as plt

In [3]:
df = pd.read_csv("../dataset/retail.csv")
df.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
144178,554441,22274,FELTCRAFT DOLL EMILY,6,24/05/2011 12:22,2.95,13213
389188,579899,22623,BOX OF VINTAGE JIGSAW BLOCKS,3,01/12/2011 08:33,5.95,15687
89377,547651,22285,HANGING HEN ON NEST DECORATION,1,24/03/2011 12:11,1.65,16904
374767,578326,21035,SET/2 RED RETROSPOT TEA TOWELS,2,23/11/2011 17:45,3.25,16343
81292,546679,23053,RECYCLED ACAPULCO MAT PINK,1,15/03/2011 16:47,8.25,13121
307912,572287,23313,VINTAGE CHRISTMAS BUNTING,3,23/10/2011 12:26,4.95,15584
343178,575628,82483,WOOD 2 DRAWER CABINET WHITE FINISH,2,10/11/2011 13:31,6.95,12844
276458,569424,23505,PLAYING CARDS I LOVE LONDON,6,04/10/2011 10:49,1.25,15529
150532,555383,22077,6 RIBBONS RUSTIC CHARM,3,02/06/2011 15:13,1.65,12517
233052,565198,82613B,"METAL SIGN,CUPCAKE SINGLE HOOK",2,01/09/2011 16:18,1.25,16729


In [23]:
df = df.loc[df["Quantity"] > 0]

Check for nan values

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 406828
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    397924 non-null  object 
 1   StockCode    397924 non-null  object 
 2   Description  397924 non-null  object 
 3   Quantity     397924 non-null  int64  
 4   InvoiceDate  397924 non-null  object 
 5   UnitPrice    397924 non-null  float64
 6   CustomerID   397924 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 24.3+ MB


In [25]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2010 08:26,2.55,17850
1,536365,71053,WHITE METAL LANTERN,6,01/12/2010 08:26,3.39,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2010 08:26,2.75,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2010 08:26,3.39,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2010 08:26,3.39,17850


In [5]:
customer_item_matrix = df.pivot_table(
    index='CustomerID',
    columns='StockCode',
    values='Quantity',
    aggfunc='sum'
)
print(customer_item_matrix.shape)

customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)

## User Based Colloborative filtering 

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

user_to_user_smatrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_to_user_smatrix.head()

,0,1,2,3,4,5,6,7,8,9,...,4362,4363,4364,4365,4366,4367,4368,4369,4370,4371
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,1.000000,0.063022,0.046130,0.047795,0.038814,0.0,0.025876,0.136641,0.094742,...,0.0,0.0,0.054656,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
2,0.0,0.063022,1.000000,0.024953,0.051709,0.027995,0.0,0.027995,0.118262,0.146427,...,0.0,0.0,0.118262,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
3,0.0,0.046130,0.024953,1.000000,0.056773,0.138314,0.0,0.030737,0.032461,0.144692,...,0.0,0.0,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
4,0.0,0.047795,0.051709,0.056773,1.000000,0.031846,0.0,0.000000,0.000000,0.033315,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


In [21]:
#Renaming index and column names

user_to_user_smatrix.columns = customer_item_matrix.index

user_to_user_smatrix['CustomerID'] = customer_item_matrix.index
user_to_user_smatrix = user_to_user_smatrix.set_index('CustomerID')
user_to_user_smatrix.head()

CustomerID,12346,12347,12348,12349,12350,12352,12353,12354,12355,12356,...,18273,18274,18276,18277,18278,18280,18281,18282,18283,18287
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
12347,0.0,1.000000,0.063022,0.046130,0.047795,0.038814,0.0,0.025876,0.136641,0.094742,...,0.0,0.0,0.054656,0.0,0.032844,0.062318,0.0,0.113776,0.109364,0.012828
12348,0.0,0.063022,1.000000,0.024953,0.051709,0.027995,0.0,0.027995,0.118262,0.146427,...,0.0,0.0,0.118262,0.0,0.000000,0.000000,0.0,0.000000,0.170905,0.083269
12349,0.0,0.046130,0.024953,1.000000,0.056773,0.138314,0.0,0.030737,0.032461,0.144692,...,0.0,0.0,0.000000,0.0,0.039014,0.000000,0.0,0.067574,0.137124,0.030475
12350,0.0,0.047795,0.051709,0.056773,1.000000,0.031846,0.0,0.000000,0.000000,0.033315,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.044866,0.000000


Top 10 Similar users to user id **12350**

In [22]:
user_to_user_smatrix.loc[12350.0].sort_values(ascending=False).head(10)

CustomerID
12350    1.000000
17935    0.183340
12414    0.181902
12652    0.175035
12814    0.171499
16754    0.171499
12791    0.171499
16692    0.171499
16426    0.166968
12475    0.161690
Name: 12350, dtype: float64

### Making recommendations for user B (id : 17935) 

In [23]:
user_to_user_smatrix.loc[12350.0].sort_values(ascending=False)
products_bought_by_A = customer_item_matrix.loc[12350.0][customer_item_matrix.loc[12350.0]>0]
print("Products Bought by A: ")
print(products_bought_by_A)

Products Bought by A: 
StockCode
20615     1
20652     1
21171     1
21832     1
21864     1
21866     1
21908     1
21915     1
22348     1
22412     1
22551     1
22557     1
22620     1
79066K    1
79191C    1
84086C    1
POST      1
Name: 12350, dtype: int64


In [28]:
products_bought_by_B = customer_item_matrix.loc[17935.0][customer_item_matrix.loc[17935.0]>0]
print("Products bought by B:")
print(products_bought_by_B)

Products bought by B:
StockCode
20657     1
20659     1
20828     1
20856     1
21051     1
21866     1
21867     1
22208     1
22209     1
22210     1
22211     1
22449     1
22450     1
22551     1
22553     1
22557     1
22640     1
22659     1
22749     1
22752     1
22753     1
22754     1
22755     1
23290     1
23292     1
23309     1
85099B    1
POST      1
Name: 17935, dtype: int64


In [31]:
products_to_recommend_to_B = set(products_bought_by_A.index) - set(products_bought_by_B.index)
print("Product Recommendation for User B ")
df.loc[df['StockCode'].isin(products_to_recommend_to_B),['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

Product Recommendation for User B 


,Description
StockCode,
21832,CHOCOLATE CALCULATOR
21915,RED HARMONICA IN BOX
22620,4 TRADITIONAL SPINNING TOPS
79066K,RETRO MOD TRAY
21864,UNION JACK FLAG PASSPORT COVER
79191C,RETRO PLASTIC ELEPHANT TRAY
21908,CHOCOLATE THIS WAY METAL SIGN
20615,BLUE POLKADOT PASSPORT COVER
20652,BLUE POLKADOT LUGGAGE TAG


## Item based collaborative filtering

In [33]:
product_similarity = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
product_similarity.columns = customer_item_matrix.T.index

product_similarity['StockCode'] = customer_item_matrix.T.index
product_similarity = product_similarity.set_index('StockCode')

In [34]:
product_similarity

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214Y,90214Z,BANK CHARGES,C2,CRUK,D,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.091287,0.0,0.000000,0.090351,0.063246,0.098907,0.095346,...,0.000000,0.0,0.000000,0.029361,0.0,0.0,0.0,0.059423,0.0,0.070057
10080,0.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.032774,0.045883,0.047836,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.017244,0.0,0.000000
10120,0.094868,0.000000,1.000000,0.115470,0.0,0.000000,0.057143,0.060000,0.041703,0.060302,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.075165,0.0,0.000000
10123C,0.091287,0.000000,0.115470,1.000000,0.0,0.000000,0.164957,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
10124A,0.000000,0.000000,0.000000,0.000000,1.0,0.447214,0.063888,0.044721,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000
DOT,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.104257,0.150756,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.000000
M,0.059423,0.017244,0.075165,0.000000,0.0,0.000000,0.075165,0.067648,0.054855,0.101983,...,0.000000,0.0,0.071307,0.000000,0.0,0.0,0.0,1.000000,0.0,0.045793


In [36]:
top_10_similar_products = list(product_similarity.loc['23166'].sort_values(ascending=False).iloc[:10].index)

### Similar Product Recommendation to Product 23166

In [38]:
df.loc[
    df['StockCode'].isin(top_10_similar_products),
    ['StockCode', 'Description']
].drop_duplicates().set_index('StockCode').loc[top_10_similar_products]

,Description
StockCode,
23166,MEDIUM CERAMIC TOP STORAGE JAR
23165,LARGE CERAMIC TOP STORAGE JAR
23167,SMALL CERAMIC TOP STORAGE JAR
22993,SET OF 4 PANTRY JELLY MOULDS
23307,SET OF 60 PANTRY DESIGN CAKE CASES
22722,SET OF 6 SPICE TINS PANTRY DESIGN
23243,SET OF TEA COFFEE SUGAR TINS PANTRY
22666,RECIPE BOX PANTRY YELLOW DESIGN
22720,SET OF 3 CAKE TINS PANTRY DESIGN
